In [1]:
import cv2
import numpy as np


In [2]:
%load_ext cython

In [17]:
%%cython -a
import numpy as np
cimport numpy as np
ctypedef np.float64_t DTYPE_t

def getPriorPatch1(np.ndarray[DTYPE_t, ndim=2] patch, np.ndarray[DTYPE_t, ndim=2] img, int h):
    cdef int k = patch.shape[0]
    cdef int n = img.shape[0]
    cdef int m = img.shape[1]
    
    cdef np.ndarray[DTYPE_t, ndim=2] num = np.zeros((k,k))
    cdef DTYPE_t denom = 0, p = 0, err, exp
    cdef np.ndarray[DTYPE_t, ndim=2] splice
        
    for i in range(k//2+1, n-k//2):
        for j in range(k//2+1, m-k//2):
            splice = img[i-k//2:i+k//2+1, j-k//2:j+k//2+1]
            err = np.sum((patch - splice)**2)
            exp = -err/(2*(h**2))
            
            p = np.exp(exp)
            num += p * splice
            denom += p
            
    return num/(denom*k*k)

In [30]:
%timeit -n3 -r3 getPriorPatch1(np.random.uniform(0,1,(11,11)), np.random.uniform(0,1, (467, 700)), 10)

4.55 s ± 55.9 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [27]:
def getPriorPatch2(patch, img, h):
    k = patch.shape[0]
    n,m = img.shape
    
    num = np.zeros((k,k))
    denom = 0
    
    cimg = im2col(img, [k,k])
    cpatch = patch.ravel()[:,None]
    
    prob = np.exp(-np.mean((cpatch - cimg)**2, axis=0)/(2*(h**2)))
    denom = np.sum(prob)
    num = cimg @ prob[:,None]
    num = num.reshape((k,k))
    
#     for i in range(k//2+1, n-k//2):
#         for j in range(k//2+1, m-k//2):
#             splice = img[i-k//2:i+k//2+1, j-k//2:j+k//2+1]
#             err = np.mean((patch - splice)**2)
#             exp = -err/(2*(h**2))
            
#             p = np.exp(exp)
#             num += p * splice
#             denom += p
            
    return num/(denom*k*k)

In [28]:
%timeit -n3 -r3 getPriorPatch2(np.random.uniform(0,1,(11,11)), np.random.uniform(0,1, (234, 350)), 10)

NameError: name 'im2col' is not defined